In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
df = pd.read_csv('C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Data\\Reg_Model_Training/model_training_data.csv')
df

,caseid,age_yrs,age_norm,age_bin_enc,gender_enc,origin_country_enc,wt_kg
0,18690315,9.00,0.499722,4,1,0,23.00
1,18690334,18.00,1.000000,5,0,1,61.80
2,18690432,14.00,0.777654,5,0,3,57.00
3,18690568,1.00,0.055031,2,1,1,10.00
4,18690589,0.25,0.013341,1,0,1,11.00
...,...,...,...,...,...,...,...
39650,24795513,16.00,0.888827,5,0,1,65.00
39651,24798894,12.00,0.666481,4,1,1,41.73
39652,24799324,16.00,0.888827,5,1,1,78.93
39653,24799665,0.17,0.008894,1,0,1,4.95


In [3]:
df['age_bin_enc'].value_counts()

age_bin_enc
5    16592
4    11448
3     4901
2     4614
1     1654
0      446
Name: count, dtype: int64

In [3]:
(df['age_bin_enc'].value_counts()/len(df))*100

age_bin_enc
5    41.840878
4    28.868995
3    12.359097
2    11.635355
1     4.170975
0     1.124701
Name: count, dtype: float64

In [4]:
#Dividing into age groups to train 3 seperate models
age_grp_1 = df[df['age_bin_enc'].isin([4,5])]
age_grp_2 = df[df['age_bin_enc'].isin([2,3])]
age_grp_3 = df[df['age_bin_enc'].isin([0,1])]

## Using the weights to train the RF model

In [5]:
#Splitting age_grp_1
X_1 = age_grp_1[['age_norm', 'age_bin_enc', 'gender_enc', 'origin_country_enc']]
y_1 = age_grp_1['wt_kg']

#Splitting age_grp_2
X_2 = age_grp_2[['age_norm', 'age_bin_enc', 'gender_enc', 'origin_country_enc']]
y_2 = age_grp_2['wt_kg']

#Splitting age_grp_3
X_3 = age_grp_3[['age_norm', 'age_bin_enc', 'gender_enc', 'origin_country_enc']]
y_3 = age_grp_3['wt_kg']

X_1.shape, y_1.shape, X_2.shape, y_2.shape, X_3.shape, y_3.shape

((28040, 4), (28040,), (9515, 4), (9515,), (2100, 4), (2100,))

In [6]:
#Splitting data
from sklearn.model_selection import train_test_split

X_1_trian, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size=0.3)

X_2_trian, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, test_size=0.3)

X_3_trian, X_3_test, y_3_train, y_3_test = train_test_split(X_3, y_3, test_size=0.3)

X_1_trian.shape, X_1_test.shape, y_1_train.shape, y_1_test.shape

((19628, 4), (8412, 4), (19628,), (8412,))

In [7]:
X_2_trian.shape, X_2_test.shape, y_2_train.shape, y_2_test.shape

((6660, 4), (2855, 4), (6660,), (2855,))

In [8]:
X_3_trian.shape, X_3_test.shape, y_3_train.shape, y_3_test.shape

((1470, 4), (630, 4), (1470,), (630,))

In [9]:
train_df_1 = pd.DataFrame(X_1_trian)
train_df_1.head()

,age_norm,age_bin_enc,gender_enc,origin_country_enc
33091,0.833241,5,1,3
4416,1.000000,5,1,1
31977,0.499722,4,0,2
33674,0.833241,5,1,1
32786,1.000000,5,1,1


In [10]:
train_df_2 = pd.DataFrame(X_2_trian)
train_df_2.head()

,age_norm,age_bin_enc,gender_enc,origin_country_enc
7509,0.166203,2,1,1
10464,0.073374,2,0,74
957,0.055031,2,1,4
37275,0.055031,2,0,25
13166,0.110617,2,0,1


In [11]:
train_df_3 = pd.DataFrame(X_3_trian)
train_df_3.head()

,age_norm,age_bin_enc,gender_enc,origin_country_enc
26627,0.017788,1,0,91
6789,0.000000,0,1,2
31154,0.017788,1,1,3
38144,0.045581,1,0,6
33787,0.036687,1,0,20


In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.class_weight import compute_sample_weight

# Compute sample weights based on age_bin_enc
sample_weights_1 = compute_sample_weight(class_weight="balanced", y=train_df_1["age_bin_enc"])
sample_weights_1

sample_weights_2 = compute_sample_weight(class_weight="balanced", y=train_df_2["age_bin_enc"])
sample_weights_2

sample_weights_3 = compute_sample_weight(class_weight="balanced", y=train_df_3["age_bin_enc"])
sample_weights_3

print(sample_weights_1), print(sample_weights_2), print(sample_weights_3)

[0.84443297 0.84443297 1.22583063 ... 0.84443297 0.84443297 0.84443297]
[1.03223807 1.03223807 1.03223807 ... 1.03223807 0.96971462 0.96971462]
[0.63968668 2.28971963 0.63968668 ... 0.63968668 0.63968668 0.63968668]


(None, None, None)

In [13]:
#Training the model with weights (age group Teens and child)
reg_1 = RandomForestRegressor(n_estimators=100, random_state=7)
reg_1.fit(X_1_trian, y_1_train, sample_weight=sample_weights_1)

RandomForestRegressor(random_state=7)

In [14]:
#Training the model with weights (age group toddler and preschooler)
reg_2 = RandomForestRegressor(n_estimators=100, random_state=7)
reg_2.fit(X_2_trian, y_2_train, sample_weight=sample_weights_2)

RandomForestRegressor(random_state=7)

In [15]:
#Training the model with weights (age group neonates and infants)
reg_3 = RandomForestRegressor(n_estimators=100, random_state=7)
reg_3.fit(X_3_trian, y_3_train, sample_weight=sample_weights_3)

RandomForestRegressor(random_state=7)

In [16]:
#Age group 1 results
preds_1 = reg_1.predict(X_1_test)
pd.DataFrame({'Actual': y_1_test, 'Prediction': preds_1, 'Diff': abs(y_1_test-preds_1)})

,Actual,Prediction,Diff
14481,55.782,57.730577,1.948577
15050,64.000,63.279651,0.720349
26247,65.000,68.981245,3.981245
24261,53.000,63.138075,10.138075
11472,20.000,24.546600,4.546600
...,...,...,...
3725,42.000,33.990435,8.009565
2794,27.700,35.610714,7.910714
11667,70.000,60.187188,9.812812
37325,54.431,62.539042,8.108042


In [17]:
#Age group 2 results
preds_2 = reg_2.predict(X_2_test)
pd.DataFrame({'Actual': y_2_test, 'Prediction': preds_2, 'Diff': abs(y_2_test-preds_2)})

,Actual,Prediction,Diff
9858,13.600,16.595396,2.995396
1881,20.860,19.038765,1.821235
8832,13.605,14.183892,0.578892
37728,17.200,14.030695,3.169305
26354,15.000,14.253532,0.746468
...,...,...,...
6545,10.000,10.152280,0.152280
27741,20.700,19.980529,0.719471
25373,15.400,12.646019,2.753981
2665,17.000,20.101148,3.101148


In [18]:
#Age group 3 results
preds_3 = reg_3.predict(X_3_test)
pd.DataFrame({'Actual': y_3_test, 'Prediction': preds_3, 'Diff': abs(y_3_test-preds_3)})

,Actual,Prediction,Diff
17789,6.45,7.533499,1.083499
2436,7.50,6.785550,0.714450
16244,7.00,5.471272,1.528728
26374,3.50,3.494291,0.005709
22723,9.00,8.887762,0.112238
...,...,...,...
10486,8.70,8.935526,0.235526
18363,6.00,5.522303,0.477697
3314,6.00,6.406980,0.406980
27004,7.90,8.464028,0.564028


In [19]:
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

r2_1 = r2_score(y_1_test, preds_1)
rmse_1 = root_mean_squared_error(y_1_test, preds_1)
mae_1 = mean_absolute_error(y_1_test, preds_1)
mape_1 = mean_absolute_percentage_error(y_1_test, preds_1)

#Displaying in a dataframe
print('Age group 1- Teens & Child evaluation')
all_fea_1 = pd.DataFrame({'R2_Score': [r2_1], 'RMSE': [rmse_1], 'MAE': [mae_1], 'MAPE': [mape_1]})
all_fea_1

Age group 1- Teens & Child evaluation


,R2_Score,RMSE,MAE,MAPE
0,0.661757,8.89837,7.152055,0.161098


In [20]:
r2_2 = r2_score(y_2_test, preds_2)
rmse_2 = root_mean_squared_error(y_2_test, preds_2)
mae_2 = mean_absolute_error(y_2_test, preds_2)
mape_2 = mean_absolute_percentage_error(y_2_test, preds_2)

#Displaying in a dataframe
print('Age group 2- Preschoolers & Toddlers evaluation')
all_fea_2 = pd.DataFrame({'R2_Score': [r2_2], 'RMSE': [rmse_2], 'MAE': [mae_2], 'MAPE': [mape_2]})
all_fea_2

Age group 2- Preschoolers & Toddlers evaluation


,R2_Score,RMSE,MAE,MAPE
0,0.624121,2.432482,1.912967,0.123613


In [21]:
r2_3 = r2_score(y_3_test, preds_3)
rmse_3 = root_mean_squared_error(y_3_test, preds_3)
mae_3 = mean_absolute_error(y_3_test, preds_3)
mape_3 = mean_absolute_percentage_error(y_3_test, preds_3)

#Displaying in a dataframe
print('Age group 3- Infants & Neonates evaluation')
all_fea_3= pd.DataFrame({'R2_Score': [r2_3], 'RMSE': [rmse_3], 'MAE': [mae_3], 'MAPE': [mape_3]})
all_fea_3

Age group 3- Infants & Neonates evaluation


,R2_Score,RMSE,MAE,MAPE
0,0.657441,1.248504,0.905006,0.1436


In [22]:
import pickle

with open("C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Scripts\\Demographic Scripts\\demog_wt_model/rf_age_grp_1.pkl", "wb") as f:
    pickle.dump(reg_1, f)

In [23]:
with open("C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Scripts\\Demographic Scripts\\demog_wt_model/rf_age_grp_2.pkl", "wb") as f:
    pickle.dump(reg_2, f)

In [24]:
with open("C:\\Users\\USER\\Desktop\\FYP\\fyp-sandbox-2\\Scripts\\Demographic Scripts\\demog_wt_model/rf_age_grp_3.pkl", "wb") as f:
    pickle.dump(reg_3, f)